In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from lightgbm import early_stopping, log_evaluation

train_path = '/content/drive/MyDrive/Copy of compressed_train_32D.csv'
test_path = '/content/drive/MyDrive/Copy of compressed_test_32D.csv'


train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)



In [ ]:

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from lightgbm import early_stopping, log_evaluation


X = train_df.drop(columns=['label']).astype(np.float32).values
y = train_df['label'].values
X_test = test_df.astype(np.float32).values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': len(np.unique(y_encoded)),
    'metric': 'multi_logloss',
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'learning_rate': 0.03,
    'num_leaves': 16,
    'max_depth': 6,
    'max_bin': 64,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.75,
    'bagging_freq': 3,
    'lambda_l1': 1.0,
    'lambda_l2': 1.0,
    'verbosity': -1,
    'random_state': 42,
    'is_unbalance': True
}

model = lgb.train(
    params,
    train_data,
    num_boost_round=100,
    valid_sets=[train_data, val_data],
    callbacks=[
        early_stopping(stopping_rounds=30),
        log_evaluation(50)
    ]
)

val_preds = np.argmax(model.predict(X_val, num_iteration=model.best_iteration), axis=1)
val_f1 = f1_score(y_val, val_preds, average='weighted')
print("Validation F1 Score (weighted):", val_f1)
print("\nClassification Report:\n")
print(classification_report(y_val, val_preds, target_names=label_encoder.classes_))

test_probs = model.predict(X_test, num_iteration=model.best_iteration)
test_preds = np.argmax(test_probs, axis=1)
test_labels = label_encoder.inverse_transform(test_preds)

submission_df = pd.DataFrame({
    'index': np.arange(len(test_labels)),
    'label': test_labels
})
submission_df.to_csv("submission_lightgbm_optimized.csv", index=False)
print("submission_lightgbm_optimized.csv saved!")


Training until validation scores don't improve for 30 rounds
[50]	training's multi_logloss: 1.58265	valid_1's multi_logloss: 1.58437
[100]	training's multi_logloss: 1.41687	valid_1's multi_logloss: 1.41914
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.41687	valid_1's multi_logloss: 1.41914
Validation F1 Score (weighted): 0.45889300253857945

Classification Report:

                         precision    recall  f1-score   support

           ARP_Spoofing       0.59      0.41      0.49      3209
                 Benign       0.90      0.98      0.94     38547
MQTT-DDoS-Connect_Flood       0.99      1.00      0.99     34607
MQTT-DDoS-Publish_Flood       0.99      0.99      0.99      5525
 MQTT-DoS-Connect_Flood       0.95      0.92      0.94      2555
 MQTT-DoS-Publish_Flood       0.99      1.00      0.99      8875
    MQTT-Malformed_Data       0.61      0.32      0.42      1026
          Recon-OS_Scan       0.45      0.03      0.05      3366
       Rec

In [ ]:
import pandas as pd

# Load the submission file
submission_df = pd.read_csv("submission_lightgbm_optimized.csv")
submission_df = submission_df.head(1048575)

# Save the truncated file
submission_df.to_csv("habiba9099.csv", index=False)

print("Truncated submission file saved as 'habiba9099.csv'.")


Truncated submission file saved as 'habiba9099.csv'.
